# Utilisation du serveur de tracking MLflow du projet

Le serveur de tracking MLFlow est accessible à l'adresse suivante : https://champi.heuzef.com

Ce notebook explique comment utiliser ce dernier.

In [1]:
# Initialisation de l'URL
mlflow_server_uri = "https://champi.heuzef.com"

# Imports et paramétrage de MLflow
from mlflow import MlflowClient
import mlflow
import setuptools

mlflow.set_tracking_uri(mlflow_server_uri)
mlflow.set_experiment("champi") # Le nom du projet

<Experiment: artifact_location='mlflow-artifacts:/103379370584144202', creation_time=1721579566179, experiment_id='103379370584144202', last_update_time=1721579566179, lifecycle_stage='active', name='champi', tags={}>

## Vérifier la disponibilité

Dans un premier temps, il faut s'assurer que le serveur est bien joignable.

In [2]:
import requests

def is_mlflow_tracking_server_available(mlflow_server_uri):
  try:
    response = requests.get(mlflow_server_uri)
    if response.status_code == 200:
      return True
    else:
      return False
  except requests.exceptions.RequestException:
    return False

if is_mlflow_tracking_server_available(mlflow_server_uri):
  print("Le serveur de tracking MLflow est disponible :", mlflow_server_uri)
else:
  print("Le serveur de tracking MLflow n'est pas disponible.")

requests.get(mlflow_server_uri)

Le serveur de tracking MLflow est disponible : https://champi.heuzef.com


<Response [200]>

# Entrainement d'un modèle pour l'exemple

Nous allons entrainer un petit modèle basique, avec Scikit-learn, pour obtenir quelques métriques qui seront enregistrés dans des variables.

In [5]:
# Imports librairies
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Import d'une database (au pif juste pour ce test)
data = pd.read_csv("https://github.com/DataScientest-Studio/MLflow/raw/main/fake_data.csv")
X = data.drop(columns=["date", "demand", "weekend", "holiday", "promo"])
y = data["demand"]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train model
params = {
    "n_estimators": 10,
    "max_depth": 10,
    "random_state": 42,
}
rf = RandomForestRegressor(**params)
rf.fit(X_train, y_train)

# Evaluate model
y_pred = rf.predict(X_val)
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_val, y_pred)
metrics = {"mae": mae, "mse": mse, "rmse": rmse, "r2": r2}

print("\nmetrics :")
print(metrics)

print("\nparams :")
print(params)


metrics :
{'mae': 130.27056867217001, 'mse': 27735.25123415424, 'rmse': 166.53903816869555, 'r2': 0.14952412556941785}

params :
{'n_estimators': 10, 'max_depth': 10, 'random_state': 42}


## Envoi des informations au serveur MLflow

Maintenant que nous avons nos resultats, nous allons donc créer une "run" et la transférer sur le serveur. 

Pour cet exemple, c'est le module mlflow.sklearn qui est utilisé. Il vous faudra bien sur utiliser celui adapté à votre outil : https://mlflow.org/docs/latest/python_api/

In [20]:
run_name = "run_test_001" # Le nom de la run, nous utiliserons notre propre nomenclature pour le projet

with mlflow.start_run(run_name=run_name) as run:
    mlflow.log_params(params)
    mlflow.log_metrics(metrics)
    mlflow.sklearn.log_model(sk_model=rf, input_example=X_val, artifact_path=run_name+"_artifacts")

Ce code executé en fin de script ou fin de notebook est finalement suffisant et assez flexible pour transférer les informations que nous souhaitons. Mais il est possible de faire encore plus simple en laissant MLflow se debrouiller avec `mlflow.autolog()`.

> https://mlflow.org/docs/latest/tracking/autolog.html

N'hésitez pas à tester des hyper-paramètres et envoyer quelques métriques à comparer sur l'interface.

# Charger le modèle le plus performant

In [7]:
import mlflow.pyfunc

champi_cnn = mlflow.pyfunc.load_model(f"models:/champi_cnn@champion")

champi_cnn

/home/heuzef/GIT/jan24_cds_mushrooms/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-01 15:18:25.979867: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 15:18:26.089889: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-01 15:18:26.138677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-01 15:18:26.233926: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-1

mlflow.pyfunc.loaded_model:
  artifact_path: heuzef_efficientnetb1_010_artifacts
  flavor: mlflow.keras
  run_id: 93ce2df782da48108f127f3e6c4adb8b